In [2]:
import torch 
import torchvision 
import torchvision.transforms as transforms
from PIL import Image, ImageFilter

In [4]:
tensor_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=tensor_transform)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=4, shuffle=False)

Files already downloaded and verified


In [6]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

net = Net()

net.load_state_dict(torch.load('cifar_net.pth'))
net.eval()

Net(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)

In [8]:
#Gaussian blur

def gaussianblur(images):
    blurred_images_batch = []
    for image in images:
        transform1 = transforms.ToPILImage()
        pil_image = transform1(image)
        blurred_image = pil_image.filter(ImageFilter.GaussianBlur(radius=0.5))
        transform2 = transforms.ToTensor()
        tensor_blurred_image = transform2(blurred_image)
        blurred_images_batch.append(tensor_blurred_image)
    blurred_images_batch = torch.stack(blurred_images_batch)
    return blurred_images_batch

In [10]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_dataloader:
        #outputs = net(images)
        preprocessed_images = gaussianblur(images)
        outputs = net(preprocessed_images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 28.08 %
